# Land subsidence prediction maps

Notebook environment to migrate .tiff files to CF compliant CoG's


In [216]:
# Optional; code formatter, installed as jupyter lab extension
# %load_ext lab_black

# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [217]:
# Import standard packages
import os
import pathlib
from pathlib import Path

import numpy as np
#import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import glob
import itertools
import json
import copy
from itertools import chain
from shapely import wkb
import json

# Import custom functionality
from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [218]:
# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    pathlib.Path().home().joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-hfda9870_3\Library\share\udunits\udunits2.xml"
    )
)

<IPython.core.display.Javascript object>

### Define drive paths

In [219]:
# Define (local and) remote drives
raw_data_dir       = p_drive.joinpath("archivedprojects", "11208003-latedeo2022", "020_InternationalDeltaPortfolio", "datasets", "00_bodemdalingsvoorspellingskaarten")
raw_data_dir       = p_drive.joinpath(r"archivedprojects\11208003-latedeo2022\020_InternationalDeltaPortfolio\datasets")  
processed_data_dir = p_drive.joinpath(r"11210264-003-delta-portal\data")  

<IPython.core.display.Javascript object>

### Read raw data

In [220]:
# Project paths & files (manual input)
raw_ms_data_dir = raw_data_dir.joinpath(r"00_bodemdalingsvoorspellingskaarten\Mild scenario")
raw_ss_data_dir = raw_data_dir.joinpath(r"00_bodemdalingsvoorspellingskaarten\Sterk scenario")

                                    
ds_ms_2020_2050_path = raw_ms_data_dir.joinpath("scenario_mild_bodemdaling_incl_ogzw_2020_2050.tif")
ds_ms_2020_2100_path = raw_ms_data_dir.joinpath("scenario_mild_bodemdaling_incl_ogzw_2020_2100.tif")
ds_ss_2020_2050_path = raw_ss_data_dir.joinpath("scenario_sterk_bodemdaling_incl_ogzw_2020_2050.tif")
ds_ss_2020_2100_path = raw_ss_data_dir.joinpath("scenario_sterk_bodemdaling_incl_ogzw_2020_2100.tif")

<IPython.core.display.Javascript object>

In [221]:
ds_ms_2020_2050 = xr.open_dataset(
    ds_ms_2020_2050_path, engine="rasterio", mask_and_scale=False
) 
ds_ms_2020_2100 = xr.open_dataset(
    ds_ms_2020_2100_path, engine="rasterio", mask_and_scale=False
) 
ds_ss_2020_2050 = xr.open_dataset(
    ds_ss_2020_2050_path, engine="rasterio", mask_and_scale=False
) 
ds_ss_2020_2100 = xr.open_dataset(
    ds_ss_2020_2100_path, engine="rasterio", mask_and_scale=False
) 

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [222]:
# Not implemented

<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [223]:
# Not implemented

<IPython.core.display.Javascript object>

### Write data to CoG

#### Single CoG test

In [224]:
VARIABLE = "subsidence"
SCENARIO = "sterk"
TIME = "2100"

ds = ds_ss_2020_2100

<IPython.core.display.Javascript object>

In [225]:
cog_dir  =  processed_data_dir.joinpath(VARIABLE, "cog")
cogs_dir =  processed_data_dir.joinpath(VARIABLE, "cogs")

cog_dir.mkdir(parents=True, exist_ok=True)
cogs_dir.mkdir(parents=True, exist_ok=True)

<IPython.core.display.Javascript object>

In [226]:
metadata_path =  processed_data_dir.joinpath(VARIABLE, "metadata_subsidence.json")

# NetCDF attribute alterations by means of metadata template
f_global    = open(metadata_path)
meta_global = json.load(f_global)

<IPython.core.display.Javascript object>

In [227]:
ds = ds.isel(band=0).drop('band')
ds.rio.write_crs("EPSG:28992")

# add all attributes (again)
for attr_name, attr_val in meta_global.items():
    if attr_name == 'PROVIDERS':
        attr_val = json.dumps(attr_val)
    if attr_name == "MEDIA_TYPE": # change media type to tiff, leave the rest as is
        attr_val = "IMAGE/TIFF"
    ds.attrs[attr_name] = attr_val

ds.attrs['Conventions'] = "CF-1.8"

output_dir  =  cog_dir.joinpath(SCENARIO)
output_dir.mkdir(parents=True, exist_ok=True)

fname = f"{TIME}.GeoTiff"

out_path = output_dir.joinpath(fname)

ds.rio.to_raster(out_path, compress="DEFLATE", driver="COG")

<IPython.core.display.Javascript object>

#### Multiple CoGs